In [34]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd drive/My Drive/hackathon

/content/drive/My Drive/hackathon


In [35]:
pwd

u'/content/drive/My Drive/hackathon'

In [0]:
import warnings
warnings.filterwarnings('ignore') # suppress import warnings

In [0]:
import os
import cv2
import tflearn
import numpy as np
import tensorflow as tf
from random import shuffle
from tqdm import tqdm 
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2


In [0]:
TRAIN_DIR = 'train1'
TEST_DIR = 'test'
IMG_SIZE = 50
LR = 1e-3
MODEL_NAME = '12skindiseasedetection-{}-{}.model'.format(LR, '2conv-basic')
tf.logging.set_verbosity(tf.logging.ERROR) # suppress keep_dims warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tensorflow gpu logs
tf.reset_default_graph()


In [44]:
def skin_label(img):
    #img.split('-',1)[0]
    skin_type = img.split('-',1)[0]
    ans = [0,0,0,0]

    if skin_type == 'acne': ans = [1,0,0,0]
    elif skin_type == 'eczema': ans = [0,1,0,0]
    elif skin_type == 'malignant': ans = [0,0,1,0]      
    elif skin_type == 'psoriasis' or 'Psoriasis': ans = [0,0,0,1]   
    return ans

def create_training_data():

    training_data = []

    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = skin_label(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])

    shuffle(training_data)
    np.save('train_data.npy', training_data)

    return training_data

def main():

    train_data = create_training_data()

    convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')

    convnet = conv_2d(convnet, 32, 3, activation='relu')
    convnet = max_pool_2d(convnet, 3)

    convnet = conv_2d(convnet, 64, 3, activation='relu')
    convnet = max_pool_2d(convnet, 3)

    convnet = conv_2d(convnet, 128, 3, activation='relu')
    convnet = max_pool_2d(convnet, 3)

    convnet = conv_2d(convnet, 32, 3, activation='relu')
    convnet = max_pool_2d(convnet, 3)

    convnet = conv_2d(convnet, 64, 3, activation='relu')
    convnet = max_pool_2d(convnet, 3)

    convnet = fully_connected(convnet, 1024, activation='relu')
    convnet = dropout(convnet, 0.8)

    convnet = fully_connected(convnet, 4, activation='softmax')
    convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

    model = tflearn.DNN(convnet, tensorboard_dir='log')

    if os.path.exists('{}.meta'.format(MODEL_NAME)):
        model.load(MODEL_NAME)
        print('Model Loaded')

    train = train_data[:-600]
    test = train_data[-50:]

    X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
    Y = [i[1] for i in train]

    test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
    test_y = [i[1] for i in test]

    model.fit({'input': X}, {'targets': Y}, n_epoch=8, validation_set=({'input': test_x}, {'targets': test_y}), snapshot_step=40, show_metric=True, run_id=MODEL_NAME)

    model.save(MODEL_NAME)

if __name__ == '__main__': main()  


Training Step: 175  | total loss: 0.73069 | time: 1.487s
| Adam | epoch: 008 | loss: 0.73069 - acc: 0.7054 -- iter: 1344/1372
Training Step: 176  | total loss: 0.70454 | time: 2.514s
| Adam | epoch: 008 | loss: 0.70454 - acc: 0.7177 | val_loss: 0.87193 - val_acc: 0.6600 -- iter: 1372/1372
--


In [29]:
import warnings
warnings.filterwarnings('ignore') # suppress import warnings

import os
import sys
import cv2
import tflearn
import numpy as np
import tensorflow as tf
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

''' <global actions> '''

IMG_SIZE = 50
LR = 1e-3
MODEL_NAME = '12diseasedetection-{}-{}.model'.format(LR, '2conv-basic')
tf.logging.set_verbosity(tf.logging.ERROR) # suppress keep_dims warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tensorflow gpu logs

''' </global actions> '''

def process_verify_data(filepath):

	verifying_data = []

	img_name = filepath.split('.')[0]
	img = cv2.imread(filepath, cv2.IMREAD_COLOR)
	img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
	verifying_data = [np.array(img), img_name]
	
	np.save('verify_data.npy', verifying_data)
	
	return verifying_data

def analysis(filepath):

	verify_data = process_verify_data(filepath)

	str_label = "Cannot make a prediction."
	status = "Error"

	tf.reset_default_graph()

	convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')

	'''
	# relu:

	Relu is used in the middle / hidden layers of the network to regularize the activation.
	It is essentialy the function: max(0, x)
	Activation should not be in negative, either it should be zero or more than that.

	# softmax: 

	Softmax is used for the output layer in multi class classification problems.
	It is essentialy the function: log(1 + e^x)
	It outputs a vector of probabilities of each class.

	'''
	
	convnet = conv_2d(convnet, 32, 3, activation='relu')
	convnet = max_pool_2d(convnet, 3)

	convnet = conv_2d(convnet, 64, 3, activation='relu')
	convnet = max_pool_2d(convnet, 3)

	convnet = conv_2d(convnet, 128, 3, activation='relu')
	convnet = max_pool_2d(convnet, 3)

	convnet = conv_2d(convnet, 32, 3, activation='relu')
	convnet = max_pool_2d(convnet, 3)

	convnet = conv_2d(convnet, 64, 3, activation='relu')
	convnet = max_pool_2d(convnet, 3)

	convnet = fully_connected(convnet, 1024, activation='relu')
	convnet = dropout(convnet, 0.8)
	
	convnet = fully_connected(convnet, 4, activation='softmax')
	convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

	model = tflearn.DNN(convnet, tensorboard_dir='log')

	if os.path.exists('{}.meta'.format(MODEL_NAME)):
		model.load(MODEL_NAME)
		print 'Model loaded successfully.'
	else:
		print 'Error: Create a model using neural_network.py first.'

	img_data, img_name = verify_data[0], verify_data[1]

	orig = img_data
	data = img_data.reshape(IMG_SIZE, IMG_SIZE, 3)

	model_out = model.predict([data])[0]

	if np.argmax(model_out) == 0: str_label = 'Acne'
	elif np.argmax(model_out) == 1: str_label = 'eczema'
	elif np.argmax(model_out) == 2: str_label = 'malignant'
	elif np.argmax(model_out) == 3: str_label = 'Psoriasis'

	if str_label =='Healthy': status = 'Healthy'
	else: status = 'Unhealthy'

	result = 'Status: ' + status + '.'
	
	if (str_label != 'Healthy'): result += '\nDisease: ' + str_label + '.'

	return result

def main():
	filepath = raw_input("Enter Image File Name:\n")
	print analysis(filepath)

if __name__ == '__main__': main()

Enter Image File Name:
acnetest1.jpg
Model loaded successfully.
Status: Unhealthy.
Disease: Psoriasis.
